# GVC participation rate

This notebook computes the trade-based and production-based GVC participation rates at the country-level and the 5-, 15-, and 35-sector levels. It uses the trade accounting framework of Borin and Mancini (2019). Results are saved as `gvcp.csv` in `data/final/`.

The trade-based GVC participation rate is defined as
```
GVCP_trade_f = (REX1 + REX2 + REX3 + REF1 + REF2) / Exports
GVCP_trade_f = (FVA + PDC1 + PDC2) / Exports
GVCP_trade = GVCP_trade_f + GVCP_trade_b
```
where sectors are broken down by export sectors.

The production-based GVC participation rate, meanwhile, is defined as
```
GVCP_prod = (DAVAX2 + REX1 + REX2 + REX3 + REF1 + REF2) / va
```
where sectors are broken down by origin sectors.

In [16]:
import numpy as np
import pandas as pd
import duckdb

In [17]:
ta = 'ta.parquet'
ta_es = 'ta-es.parquet'
ta_os = 'ta-os.parquet'
summary = 'summary.csv'
outputfile = 'gvcp.csv'

# ta = 'ta62.parquet'
# ta_es = 'ta62-es.parquet'
# ta_os = 'ta62-os.parquet'
# summary = 'summary62.csv'
# outputfile = 'gvcp62.csv'

# ta = 'ta_jun2023.parquet'
# ta_es = 'ta-es_jun2023.parquet'
# ta_os = 'ta-os_jun2023.parquet'
# summary = 'summary_jun2023.csv'
# outputfile = 'gvcp_jun2023.csv'

## Load and process data

### Breakdown by export sectors

In [18]:
df_es = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta_es}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta_es}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta_es}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_es['GVC_trade_f'] = df_es['REX1'] + df_es['REX2'] + df_es['REX3'] + df_es['REF1'] + df_es['REF2']
df_es['GVC_trade_b'] = df_es['FVA'] + df_es['PDC1'] + df_es['PDC2']
df_es['GVC_trade'] = df_es['GVC_trade_f'] + df_es['GVC_trade_b']
df_es['t'] = df_es['t'].astype(int)
df_es = df_es[['t', 's', 'agg', 'i', 'Exports', 'GVC_trade_f', 'GVC_trade_b', 'GVC_trade']]

### Breakdown by origin sectors

In [19]:
df_os = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_os['GVC_prod'] = df_os['DAVAX2'] + df_os['REX1'] + df_os['REX2'] + df_os['REX3'] + df_os['REF1'] + df_os['REF2']
df_os['t'] = df_os['t'].astype(int)
df_os = df_os[['t', 's', 'agg', 'i', 'GVC_prod']]

### Value added

In [20]:
va = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, SUM(va) AS va, 
    FROM read_csv_auto('../data/final/{summary}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, SUM(va) AS va, 
    FROM read_csv_auto('../data/final/{summary}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL

    (SELECT t, s, 15 AS agg, i15 AS i, SUM(va) AS va, 
    FROM read_csv_auto('../data/final/{summary}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)

    UNION ALL

    (SELECT t, s, 35 AS agg, i, SUM(va) AS va, 
    FROM read_csv_auto('../data/final/{summary}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

In [21]:
df_os = pd.merge(df_os, va)

## Consolidate and save

In [22]:
df = pd.merge(df_es, df_os)
df['GVCP_trade_f'] = df['GVC_trade_f'] / df['Exports']
df['GVCP_trade_b'] = df['GVC_trade_b'] / df['Exports']
df['GVCP_trade'] = df['GVC_trade'] / df['Exports']
df['GVCP_prod'] = df['GVC_prod'] / df['va']

df = df[[
    't', 's', 'agg', 'i', 'Exports', 'va',
    'GVC_trade_f', 'GVC_trade_b', 'GVC_trade', 'GVC_prod',
    'GVCP_trade_f', 'GVCP_trade_b', 'GVCP_trade', 'GVCP_prod'
]]

df.to_csv(f'../data/final/{outputfile}', index=False)

In [23]:
df

,t,s,agg,i,Exports,va,GVC_trade_f,GVC_trade_b,GVC_trade,GVC_prod,GVCP_trade_f,GVCP_trade_b,GVCP_trade,GVCP_prod
0,2017,1,0,0,307080.681967,1.333367e+06,79461.553936,42639.546595,122101.100531,226376.419796,0.258764,0.138855,0.397619,0.169778
1,2017,2,0,0,201266.630265,3.811415e+05,42872.231145,70199.936072,113072.167216,83955.732939,0.213012,0.348791,0.561803,0.220274
2,2017,3,0,0,424418.632561,4.550668e+05,65010.218835,224150.725071,289160.943905,132095.410861,0.153175,0.528136,0.681311,0.290277
3,2017,4,0,0,39223.820989,5.326033e+04,7353.689482,15200.185953,22553.875436,16386.390843,0.187480,0.387524,0.575005,0.307666
4,2017,5,0,0,258208.698095,1.922583e+06,51817.604375,35197.431960,87015.036334,155391.127964,0.200681,0.136314,0.336995,0.080824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,2022,73,35,31,18213.907000,3.556324e+05,3651.368957,3463.219850,7114.588807,15137.038259,0.200471,0.190142,0.390613,0.042564
24524,2022,73,35,32,13170.327289,2.556576e+05,1371.397409,2285.260489,3656.657898,6235.026000,0.104128,0.173516,0.277644,0.024388
24525,2022,73,35,33,9870.339282,1.627225e+05,610.135130,2484.601156,3094.736286,5990.617412,0.061815,0.251724,0.313539,0.036815
24526,2022,73,35,34,11296.663394,1.541211e+05,1793.894818,2925.590377,4719.485195,20644.725650,0.158799,0.258978,0.417777,0.133951
